In [1]:
import pandas as pd
# import dill as pickle


In [2]:
# To find the number of substances tested
with open('chemical_notation_data/substances_smiles.txt', 'r') as f:
    data = f.readlines()
    i = 1
    for line in data:
        words = line.split()
        i += 1
    print i 

# Expected: 392719

392719


In [3]:
# To find the number of compounds tested
with open('chemical_notation_data/compounds_inchi.txt', 'r') as f:
    data = f.readlines()
    i = 1
    for line in data:
        words = line.split()
        i += 1
    print i 

# Expected: 389561

389561


In [4]:
def create_dict(filename, mol):
    """
    Returns dictionary of sample ID and molecular notation
    
    Inputs: filename, mol
    filename - path to file containing molecular notation indexed by sample ID
    mol - type of molecular notation
    Input types: str, str
    
    Outputs: dictionary of molecular notation indexed by sample ID
    Output types: Python dictionary
    """
    with open(filename, 'r') as f:
        # Reads the file line by line
        data = f.readlines()
        # Null dictionary
        df = (dict([]))
        for line in data[:100]:
            # Splits the line into it's key and molecular string  
            words = line.split()
            if mol == 'smiles':
                z = (dict([(int(words[0]), [words[1]])]))
            elif mol == 'inchi':
                # This removes the 'InChI=' prefix to the InChI string
                z = (dict([(int(words[0]), [words[1][6:]])]))
            else:
                print 'Invalid molecular notation. Choose from smiles or inchi.'
            # Appending dictionary            
            df.update(z)
        return df

In [5]:
# The SMILES and InChI logs of the same material have identical keys 
# Creating and joining the SMILES and InChI dictionaries along the same index
# dict_substances = {key: value + create_dict('chemical_notation_data/substances_inchi.txt', 'inchi')[key] for key, value in 
                 #create_dict('chemical_notation_data/substances_smiles.txt', 'smiles').iteritems()}
# dict_substances_active = {key: value + create_dict('chemical_notation_data/substances_active_inchi.txt', 'inchi')[key] for key, value in 
                 #create_dict('chemical_notation_data/substances_active_smiles.txt', 'smiles').iteritems()}

dict_compounds = {key: value + create_dict('chemical_notation_data/compounds_inchi.txt', 'inchi')[key] for key, value in 
                create_dict('chemical_notation_data/compounds_smiles.txt','smiles').iteritems()}
dict_compounds_active = {key: value + create_dict('chemical_notation_data/compounds_active_inchi.txt', 'inchi')[key] for key, value in 
                create_dict('chemical_notation_data/compounds_active_smiles.txt', 'smiles').iteritems()}

In [6]:
def create_dataframe(filename, mol):
    """
    Returns Pandas dataframe of sample ID and molecular notation
    
    Inputs: filename, mol
    filename - path to file containing molecular notation indexed by sample ID
    mol - type of molecular notation
    Input types: str, str
    
    Outputs: dataframe of molecular notation indexed by sample ID
    Output types: Pandas DataFrame
    """
    with open(filename, 'r') as f:
        # Reads the file line by line
        data = f.readlines()
        # Null dataframe
        df = []
        for line in data[:100]:
            # Splits the line into it's key and molecular string  
            words = line.split()
            if mol == 'smiles':
                z = [int(words[0]), words[1]]
            elif mol == 'inchi':
                # This removes the 'InChI=' prefix to the InChI string
                z = [int(words[0]), words[1][6:]]
            else:
                print 'Invalid molecular notation. Choose from smiles or inchi.'
            # Appending dictionary            
            df.append(z)
        df = pd.DataFrame(df)
        df.columns = ['ID', mol.upper()]
        # df = df.set_index('ID')
        return df

In [7]:
# The SMILES and InChI logs of the same material have identical indices 
# Creating and joining the SMILES and InChI dataframes along the same index

#df_substances_smiles = create_dataframe('chemical_notation_data/substances_smiles.txt', 'smiles')
#df_substances_inchi = create_dataframe('chemical_notation_data/substances_inchi.txt', 'inchi')

df_compounds_smiles = create_dataframe('chemical_notation_data/compounds_smiles.txt', 'smiles')
df_compounds_inchi = create_dataframe('chemical_notation_data/compounds_inchi.txt','inchi')

#df_substances = pd.concat([df_substances_smiles, df_substances_inchi['INCHI']], axis=1).rename(columns = {'ID':'SID'})
df_compounds = pd.concat([df_compounds_smiles, df_compounds_inchi['INCHI']], axis=1).rename(columns = {'ID':'CID'})

In [8]:
"""file_Name = "testfile"
# open the file for writing
fileObject = open(file_Name,'wb') 
a = open('activity_data/AID_743255_datatable_part1.xlsx', 'r')

# this writes the object a to the
# file named 'testfile'
pickle.dump(a,fileObject)   

# here we close the fileObject
fileObject.close()"""

'file_Name = "testfile"\n# open the file for writing\nfileObject = open(file_Name,\'wb\') \na = open(\'activity_data/AID_743255_datatable_part1.xlsx\', \'r\')\n\n# this writes the object a to the\n# file named \'testfile\'\npickle.dump(a,fileObject)   \n\n# here we close the fileObject\nfileObject.close()'

In [9]:
activity = pd.read_csv('activity_data/AID_743255_datatable.csv')
for i in range(5):
    activity = activity.drop(i, axis=0)
activity = activity.drop(['PUBCHEM_ACTIVITY_URL', 'PUBCHEM_ASSAYDATA_COMMENT', 'Potency', 'Efficacy','Analysis Comment', 
                     'Curve_Description','Fit_LogAC50','Fit_HillSlope','Fit_R2','Fit_InfiniteActivity','Fit_ZeroActivity',
                     'Fit_CurveClass', 'Excluded_Points', 'Compound QC'], axis=1)
activity.reset_index(['PUBCHEM_RESULT_TAG'], drop=True)
#activity.Index.names = ['#']

C:\Users\pearl\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (0,8,9,10,11,13,14,15,16,17,18,20,21,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,PUBCHEM_RESULT_TAG,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,Phenotype,Activity_Score,Max_Response,Activity at 0.457 uM,Activity at 2.290 uM,Activity at 11.40 uM,Activity at 57.10 uM
0,1,842121,6603008,Inactive,0,Inactive,0,-5.4019,-3.0502,-6.4209,-5.4822,-5.4019
1,2,842122,6602571,Inactive,0,Inactive,0,-3.3968,-11.07,-13.6332,-9.3508,-3.3968
2,3,842123,6602616,Inactive,0,Inactive,0,13.7444,7.1989,3.1215,3.4647,13.7444
3,4,842124,644371,Inactive,0,Inactive,0,-7.0279,0.1482,-10.6303,-2.2984,-7.0279
4,5,842125,6603132,Inactive,0,Inactive,0,-23.6708,-24.1881,-25.2434,-27.6096,-23.6708
5,6,842126,2850911,Inactive,0,Inactive,0,-0.3514,-5.3005,0.3418,0.4484,-0.3514
6,7,842127,6603374,Inactive,0,Inactive,0,-12.3013,-16.037,-17.5522,-15.6999,-12.3013
7,8,842128,6603127,Inactive,0,Inactive,0,7.1004,-2.1207,15.6887,9.8457,7.1004
8,9,842129,644380,Inactive,0,Inactive,0,9.5188,-6.1754,-1.9454,2.5167,9.5188
9,10,842130,2900108,Inactive,0,Inactive,0,4.6671,-7.3583,4.1302,-2.9418,4.6671


In [11]:
print df_compounds

         CID                                             SMILES  \
0   56643186    CCOC(=O)N1CCC(CC1)NCCNC(=O)C2=CC(=C(C=C2)F)F.Cl   
1   56643185       CCOC(=O)N1CCC(CC1)NCCNC(=O)C2=CC(=CC=C2)F.Cl   
2   56643184       CCOC(=O)N1CCC(CC1)NCCNC(=O)C2=CC=C(C=C2)F.Cl   
3   56642972  CC(C)(C)S(=O)(=O)N1C[C@H]/2CC/C=C\C/C=C2/[C@@H...   
4   56642971    CCOC1=C(C=CC(=C1)/C=N/NC2=NN=C(N2N)C)OC(=O)C.Cl   
5   56642970  C1COCCN1C2=CC=C(C=C2)/C=N/NC(=O)C3COC4=CC=CC=C4O3   
6   56642969   CC1=NN=C(N1N)N/N=C/C2=CC=C(O2)C3=CC=C(C=C3)Br.Cl   
7   56642968  CCC/C(=N\S(=O)(=O)C1=CC=C(C=C1)N=NN(C)C)/[O-]....   
8   56642967      C1CCCN(CC1)C(=O)C[N+]2=CC=CC(=C2)/C=N/O.[Cl-]   
9   56642964  CCO.CC(=O)OC1CC\2(C(CC/C2=N/O)C3C1C4=C(CC3)C=C...   
10  56642963        CC(=O)OC1=C(C=C(C=C1)/C=N\NC2=NN=CN2N)OC.Cl   
11  56642962                    CN(C1=NCCN1)/N=C/C2=CC=NC=C2.Br   
12  56642961  CCN1CCN(CC1)CC2=C(N=NN2C3=NON=C3N)C(=O)N/N=C(\...   
13  56642960                     C1CN=C(N1)N/N=C/C2=CC=NC=C2.C